In [3]:
import numpy as np
import pandas as pd
import math

In [1]:
def calculate_t(x, y, lambd, mil, p):
    if x == 0 and y == 0:
        return 1 - lambd*mil*p
    elif x == 0 and y == 1:
        return 1 + lambd*p
    elif x == 1 and y == 0:
        return 1 + mil*p
    elif x == 1 and y == 1:
        return 1 - p
    else: 
        return 1

In [4]:
def score_probability(x, y, home_attack, home_defence, away_attack, away_defence, home_advantage, p):
    lambd = home_attack * away_defence * home_advantage
    mil = away_attack * home_defence
    t = calculate_t(x, y, lambd, mil, p)
    poisson_x = (lambd**x * np.exp(-lambd))/math.factorial(x)
    poisson_y = (mil**y * np.exp(-mil))/math.factorial(y)
    return t*poisson_x*poisson_y    

In [12]:
df_attack = pd.read_csv('./data/attacking_scores.csv', index_col='team')
df_defence = pd.read_csv('./data/defending_scores.csv', index_col='team')
df_home_advantage = pd.read_csv('./data/home_advantage.csv', index_col='parameter')

In [26]:
def get_probability_array(home_team, away_team):
    prob_array = np.zeros((9,9))
    total_probability = 0
    home_attack = df_attack.loc[home_team].attacking_score
    away_attack = df_attack.loc[away_team].attacking_score
    home_defence = df_defence.loc[home_team].defending_score
    away_defence = df_defence.loc[away_team].defending_score
    home_advantage = df_home_advantage.loc['home_advantage'].value
    p = df_home_advantage.loc['p'].value

    # Get probability of each scoreline up to 9 goals
    for i in range(9):
        for j in range(9):
            prob = score_probability(i, j, home_attack, home_defence, away_attack, away_defence, home_advantage, p)
            prob_array[i][j] = prob
            total_probability += prob
            
    prob_array /= total_probability # Normalise probabilities
    return prob_array

In [27]:
prob_array = get_probability_array('Manchester City', 'Brentford')

In [31]:
def match_odds(prob_array):
    home_win = 0
    draw = 0
    away_win = 0
    rows, cols = prob_array.shape
    for i in range(rows):
        for j in range(cols):
            if i > j:
                home_win += prob_array[i][j]
            elif j > i:
                away_win += prob_array[i][j]
            else: 
                draw += prob_array[i][j]
    home_win_odds = np.round(1/home_win,2)
    draw_odds = np.round(1/draw,2)
    away_win_odds = np.round(1/away_win,2)
    return [home_win_odds, draw_odds, away_win_odds]

In [40]:
def score_odds(prob_array):
    return pd.DataFrame(np.round(1/prob_array,2))

In [44]:
def over_under_odds(prob_array, goals):
    over = 0
    under = 0
    rows, cols = prob_array.shape
    for i in range(rows):
        for j in range(cols):
            if i+j > goals:
                over += prob_array[i][j]
            else:
                under += prob_array[i][j]
    over_odds = np.round(1/over,2)
    under_odds = np.round(1/under,2)
    return [over_odds, under_odds]

In [49]:
def both_to_score(prob_array):
    both_score = 0
    not_both_score = 0
    rows, cols = prob_array.shape
    for i in range(rows):
        for j in range(cols):
            if i == 0 or j == 0:
                not_both_score += prob_array[i][j]
            else: 
                both_score += prob_array[i][j]
    both_score_odds = np.round(1/both_score,2)
    not_both_score_odds = np.round(1/not_both_score,2)
    return [both_score_odds, not_both_score_odds]

In [51]:
def result_both_to_score(prob_array):
    home_win_both_score = 0
    home_win_not_both_score = 0
    draw_both_score = 0
    draw_not_both_score = 0
    away_win_both_score = 0
    away_win_not_both_score = 0
    rows, cols = prob_array.shape
    for i in range(rows):
        for j in range(cols):
            if i > j:
                if j == 0:
                    home_win_not_both_score += prob_array[i][j]
                else:
                    home_win_both_score += prob_array[i][j]
            elif j > i:
                if i == 0:
                    away_win_not_both_score += prob_array[i][j]
                else: 
                    away_win_both_score += prob_array[i][j]
            else:
                if j == 0:
                    draw_not_both_score += prob_array[i][j]
                else:
                    draw_both_score += prob_array[i][j]
    home_win_both_score_odds = np.round(1/home_win_both_score,2)
    home_win_not_both_score_odds = np.round(1/home_win_not_both_score,2)
    draw_both_score_odds = np.round(1/draw_both_score,2)
    draw_not_both_score_odds = np.round(1/draw_not_both_score,2)
    away_win_both_score_odds = np.round(1/away_win_both_score,2)
    away_win_not_both_score_odds = np.round(1/away_win_not_both_score,2)
    return [home_win_both_score_odds, home_win_not_both_score_odds, draw_both_score_odds, 
            draw_not_both_score_odds, away_win_both_score_odds, away_win_not_both_score_odds]